<a href="https://colab.research.google.com/github/SARA3SAEED/LLM-2/blob/main/36_Module_1a_Advanced_LLMs_semantic_cache_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**If you use our code, please cite:**

@misc{2024<br>
  title = {Semantic Cache from Scratch},<br>
  author = {Hamza Farooq, Darshil Modi, Kanwal Mehreen, Nazila Shafiei},<br>
  keywords = {Semantic Cache},<br>
  year = {2024},<br>
  copyright = {APACHE 2.0 license}<br>
}

In [1]:
!pip install -U faiss-cpu sentence_transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
import faiss
import sqlite3
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from pprint import pprint

In [ ]:
import faiss
import sqlite3
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from pprint import pprint


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



# Traversaal Ares API Overview

Traversaal Ares API is a cutting-edge solution designed to provide real-time search results generated from user queries. Leveraging advanced Large Language Models (LLMs), Ares connects to the internet to deliver accurate and factual information, including relevant URLs for reference. This API is tailored for speed and efficiency, providing lightning-fast search results within 3-4 seconds. Currently available for free during the beta phase, with priced solutions coming soon.

## Key Features:
- **Real-time Search Results:** Ares API offers unparalleled speed in generating search results.
- **Internet Connectivity:** Connects to the internet to fetch the latest and most accurate information.
- **Lightning-Fast Response:** Delivers search results with URLs in 3-4 seconds.
- **Free Beta Access:** Available for free during for the first 100 calls
- **Factual and Accurate:** Ensures the information provided is accurate and supported by relevant references. [Can make mistakes though]

## Getting Started:
To access the Ares API, sign up at [api.traversaal.ai](https://api.traversaal.ai) and refer to the usage documentation at [docs.traversaal.ai](https://docs.traversaal.ai/docs/intro).

Experience the future of AI-driven search with Traversaal Ares API!


In [ ]:
from google.colab import userdata
import requests

def make_prediction(data):
    url = "https://api-ares.traversaal.ai/live/predict"
    headers = {
        "x-api-key": userdata.get('ARES_KEY'),
        "content-type": "application/json"
    }

    payload = {"query": data}

    try:
        response = requests.post(url, json=payload, headers=headers)

        if response.status_code == 200:
            # The request was successful
            print("Request was successful.")
            # If the response contains JSON data, you can parse it using response.json()
            try:
                json_data = response.json()
                #print("Parsed JSON data:", json_data)
                return json_data
            except ValueError:
                print("No JSON data in the response.")
                return None
        else:
            # The request was not successful, handle the error
            print(f"Request failed with status code {response.status_code}.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None

# Example usage



In [ ]:
response=make_prediction(['Events happening in London this week. '])

Request was successful.


In [ ]:
response

{'data': {'response_text': 'Here are some events happening in London this week:\n\n1. UEFA Euros: Enjoy the excitement of the UEFA Euros football tournament taking place in London this week. Check out the official website for match schedules and ticket information.\n\n2. Wimbledon: Experience the world-renowned Wimbledon tennis tournament, featuring top players from around the globe. Visit the official Wimbledon website for details on matches and ticket availability.\n\n3. The Secret Garden: Immerse yourself in the enchanting world of "The Secret Garden" at a theater in London. Check out Time Out\'s website for more information on showtimes and venues.\n\n4. Wireless Festival: Dance to the beats of your favorite artists at the Wireless Festival. This music event showcases a lineup of talented performers. Visit Time Out\'s website for details on the festival and how to get tickets.\n\nPlease note that the provided information is based on available sources and may be subject to change.',

In [ ]:
pprint(response['data']['response_text'])

('Here are some events happening in London this week:\n'
 '\n'
 '1. UEFA Euros: Enjoy the excitement of the UEFA Euros football tournament '
 'taking place in London this week.\n'
 '\n'
 '2. Wimbledon: Experience the world-renowned Wimbledon tennis tournament, '
 'where you can watch top tennis players compete for glory.\n'
 '\n'
 '3. Barbie at the Design Museum: Visit the Design Museum to explore the '
 'fascinating exhibition dedicated to Barbie, showcasing her impact on culture '
 'and design.\n'
 '\n'
 '4. The Constituent: Attend the theatrical production of "The Constituent," a '
 'thought-provoking play that explores political themes and societal issues.\n'
 '\n'
 "5. London Film & Comic Con 2024: If you're a fan of films, comics, and pop "
 "culture, don't miss the London Film & Comic Con happening this weekend at "
 'Olympia London.\n'
 '\n'
 'Please note that the information provided is based on available sources and '
 'may be subject to change. For more details and additiona

In [ ]:
response['data']['web_url']

['https://www.timeout.com/london/things-to-do/things-to-do-in-london-this-week',
 'https://www.timeout.com/london',
 'https://www.eventbrite.com/d/united-kingdom--london/events--this-weekend/',
 'https://www.visitlondon.com/things-to-do/whats-on/special-events/london-events-calendar',
 'https://www.eventbrite.com/d/united-kingdom--london/events--this-week/',
 'https://londonist.com/things-to-do-in-london-this-week',
 'https://londontheinside.com/whats-on-this-week/',
 'https://www.londontourism.ca/events',
 'https://www.designmynight.com/london/whats-on/whats-on-this-week-in-london',
 'https://www.londontourism.ca/events/events-this-week']

Instead of using an LLM endpoint, we will be using Ares API for retrieval and generation, however you can replace is with your own rag function in 'generate answer' function

In [ ]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

class SemanticCaching:
    def __init__(self, json_file='cache.json'):
        # Initialize Faiss index with Euclidean distance
        self.index = faiss.IndexFlatL2(768)  # Use IndexFlatL2 with Euclidean distance
        if self.index.is_trained:
            print('Index trained')

        # Initialize Sentence Transformer model
        self.encoder = SentenceTransformer('all-mpnet-base-v2')


        # Uncomment the following lines to use DialoGPT for question generation
        # self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
        # self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

        # Set Euclidean distance threshold
        self.euclidean_threshold = 0.3
        self.json_file = json_file
        self.load_cache()

    def load_cache(self):
        # Load cache from JSON file, creating an empty cache if the file is not found
        try:
            with open(self.json_file, 'r') as file:
                self.cache = json.load(file)
        except FileNotFoundError:
            self.cache = {'questions': [], 'embeddings': [], 'answers': [], 'response_text': []}

    def save_cache(self):
        # Save the cache to the JSON file
        with open(self.json_file, 'w') as file:
            json.dump(self.cache, file)

    def ask(self, question: str) -> str:
        # Method to retrieve an answer from the cache or generate a new one
        start_time = time.time()
        try:
            l = [question]
            embedding = self.encoder.encode(l)

            # Search for the nearest neighbor in the index
            D, I = self.index.search(embedding, 1)

            if D[0] >= 0:
                if I[0][0] != -1 and D[0][0] <= self.euclidean_threshold:
                    row_id = int(I[0][0])
                    print(f'Found cache in row: {row_id} with score {1 - D[0][0]}') #score inversed to show similarity
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    print(f"Time taken: {elapsed_time} seconds")
                    return self.cache['response_text'][row_id]

            # Handle the case when there are not enough results or Euclidean distance is not met
            answer, response_text = self.generate_answer(question)

            self.cache['questions'].append(question)
            self.cache['embeddings'].append(embedding[0].tolist())
            self.cache['answers'].append(answer)
            self.cache['response_text'].append(response_text)

            self.index.add(embedding)
            self.save_cache()
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Time taken: {elapsed_time} seconds")

            return response_text
        except Exception as e:
            raise RuntimeError(f"Error during 'ask' method: {e}")

    def generate_answer(self, question: str) -> str:
        # Method to generate an answer using a separate function (make_prediction in this case)
        try:
            result = make_prediction([question])
            response_text = result['data']['response_text']

            return result, response_text
        except Exception as e:
            raise RuntimeError(f"Error during 'generate_answer' method: {e}")


In [ ]:
cache = SemanticCaching()



Index trained


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
question1 = "What is the capital of France?"
answer1 = cache.ask(question1)
print(answer1)

# Question not seen before, generates answer from LLM

question2 = "Who is the CEO of Apple?"
answer2 = cache.ask(question2)
print(answer2)

# Stores question2, embedding and answer2 in cache

question3 = "Who is the CEO of Facebook?"
answer3 = cache.ask(question3)
print(answer3)

# Finds question2 is similar above threshold
# Returns cached answer2 instead of generating new answer

Request was successful.
Time taken: 4.441723108291626 seconds
The capital of France is Paris.
Request was successful.
Time taken: 3.3008177280426025 seconds
The CEO of Apple is Tim Cook. He has been serving as the CEO since 2011 and also serves on the company's board of directors. You can find more information about Tim Cook and his role at Apple on the Apple Leadership page [^1^]. Tim Cook's profile on Wikipedia provides additional details about his tenure as CEO of Apple [^2^]. Forbes also recognizes Tim Cook as the CEO of Apple, highlighting the company's success under his leadership [^3^]. If you are interested in Tim Cook's social media presence, you can follow him on Twitter [@tim_cook] [^4^]. Additionally, Business Insider provides insights into Tim Cook's compensation and net worth [^5^][^8^].
Request was successful.
Time taken: 3.2995970249176025 seconds
The CEO of Facebook is Mark Elliot Zuckerberg. He is an American businessman who co-founded the social media service Faceboo

In [ ]:
answer4 = cache.ask("What is the Capital of India")
print(answer4)

Request was successful.
Time taken: 4.233320713043213 seconds
The capital of India is New Delhi. It is located in the north-central part of the country on the west bank of the Yamuna River. New Delhi is the seat of all three branches of the Government of India. It is a part of the National Capital Territory of Delhi (NCT) and is considered the national capital of India.


In [ ]:
answer4 = cache.ask("Can you tell me what is the Capital of India")
print(answer4)

Found cache in row: 3 with score 0.8059847354888916
Time taken: 0.10534310340881348 seconds
The capital of India is New Delhi. It is located in the north-central part of the country on the west bank of the Yamuna River. New Delhi is the seat of all three branches of the Government of India. It is a part of the National Capital Territory of Delhi (NCT) and is considered the national capital of India.


In [ ]:
print(cache.ask('Who is the CEO of Facebook?'))

Found cache in row: 2 with score 1.0
Time taken: 0.09782648086547852 seconds
The CEO of Facebook is Mark Elliot Zuckerberg. He is an American businessman who co-founded the social media service Facebook and its parent company Meta Platforms (formerly Facebook, Inc.). Mark Zuckerberg currently serves as the chairman, chief executive officer, and controlling shareholder of Meta Platforms. You can find more information about Mark Zuckerberg on his Wikipedia page [^1^]. Additionally, you can also visit his Facebook profile [^2^], Meta's official website [^3^], Meta Investor Relations [^4^], his Instagram profile [^5^], his LinkedIn profile [^6^], and a YouTube video featuring Mark Zuckerberg [^7^].


In [ ]:
print(cache.ask('Who is the current CEO of Google?'))

Request was successful.
Time taken: 5.717427730560303 seconds
The current CEO of Google is Sundar Pichai. Sundar Pichai is also the CEO of Alphabet Inc., the parent company of Google. He serves on Alphabet's Board of Directors. Under his leadership, Google has been focused on developing products and services powered by the latest advances in AI that offer help in moments big and small. You can find more information about Sundar Pichai on the Google Blog [^1^], Wikipedia [^2^], Instagram [^3^], Britannica Money [^4^], Business Insider [^5^], LinkedIn [^6^], ProductPlan [^7^], Twitter [^8^], and Business Roundtable [^9^].

Sources:
1. [Sundar Pichai | Google Blog](https://blog.google/authors/sundar-pichai/)
2. [Sundar Pichai - Wikipedia](https://en.wikipedia.org/wiki/Sundar_Pichai)
3. [Sundar Pichai (@sundarpichai) - Instagram](https://www.instagram.com/sundarpichai/?hl=en)
4. [Sundar Pichai | Biography, Google, & Facts | Britannica Money](https://www.britannica.com/money/Sundar-Pichai)


In [ ]:
print(cache.ask('Is Sundar Pichai the CEO of Google?'))

Request was successful.
Time taken: 8.281602144241333 seconds
Yes, Sundar Pichai is the CEO of Google. According to Wikipedia, Sundar Pichai, whose full name is Pichai Sundararajan, is an Indian-born American business executive. He has been serving as the chief executive officer (CEO) of Alphabet Inc. and its subsidiary Google. You can find more information about Sundar Pichai on his Wikipedia page at the following link: [Sundar Pichai - Wikipedia](https://en.wikipedia.org/wiki/Sundar_Pichai).

Additionally, Sundar Pichai's LinkedIn profile confirms that he is the CEO of Google and Alphabet. His focus is on organizing the world's information and making it universally accessible and useful, as well as building great products. You can find his LinkedIn profile at the following link: [Sundar Pichai - CEO - Google - LinkedIn](https://www.linkedin.com/in/sundarpichai).

Sundar Pichai is also active on Instagram, where he shares updates and insights. His Instagram handle is @sundarpichai, an

In [ ]:
print(cache.ask('Best local food spots in Edinburgh for a couple?'))

Request was successful.
Time taken: 6.557676076889038 seconds
Here are some of the best local food spots in Edinburgh for a couple:

1. Dine: This European and British restaurant offers a romantic atmosphere and has received positive reviews. You can find more information [here](https://www.tripadvisor.com/Restaurants-g186525-zfp3-Edinburgh_Scotland.html).

2. The Witchery By The Castle: Known for its romantic ambiance, this restaurant is located near the castle and offers a unique dining experience. You can find more information [here](https://www.wayfaringkiwi.com/romantic-restaurants-in-edinburgh/).

3. The Voodoo Rooms: This restaurant offers a blend of dining, cocktails, and live music, creating a vibrant and romantic atmosphere. You can find more information [here](https://www.wayfaringkiwi.com/romantic-restaurants-in-edinburgh/).

4. Bluebird Cafe: A local favorite, this cafe offers delicious food with a cozy and intimate setting. You can find more information [here](https://www

In [ ]:
print(cache.ask('Best local food spots in Edinburgh?'))

Found cache in row: 6 with score 0.8507777154445648
Time taken: 0.09998726844787598 seconds
Here are some of the best local food spots in Edinburgh for a couple:

1. Dine: This European and British restaurant offers a romantic atmosphere and has received positive reviews. You can find more information [here](https://www.tripadvisor.com/Restaurants-g186525-zfp3-Edinburgh_Scotland.html).

2. The Witchery By The Castle: Known for its romantic ambiance, this restaurant is located near the castle and offers a unique dining experience. You can find more information [here](https://www.wayfaringkiwi.com/romantic-restaurants-in-edinburgh/).

3. The Voodoo Rooms: This restaurant offers a blend of dining, cocktails, and live music, creating a vibrant and romantic atmosphere. You can find more information [here](https://www.wayfaringkiwi.com/romantic-restaurants-in-edinburgh/).

4. Bluebird Cafe: A local favorite, this cafe offers delicious food with a cozy and intimate setting. You can find more 

In [ ]:
print(cache.ask('Best local food spots in London?'))

Request was successful.
Time taken: 5.182124853134155 seconds
Based on the information provided, here are some of the best local food spots in London:

1. Casa Pastor in King's Cross
2. Tayyabs in Whitechapel
3. Oklava in Shoreditch
4. Bright in Hackney
5. Fish, Wings & Tings in Brixton
6. Kudu in Peckham
7. Petersham Nurseries Café and Tearoom in Richmond
8. Maggie Jones's in Kensington

You can find more information about these restaurants and other local food spots in London on the [Visit London website](https://www.visitlondon.com/things-to-do/food-and-drink/restaurant/local-restaurants).

Please note that this information is based on the available sources and may be subject to change.


In [ ]:
print(cache.ask('Best local food spots in London?'))

Found cache in row: 7 with score 1.0
Time taken: 0.11299681663513184 seconds
Based on the information provided, here are some of the best local food spots in London:

1. Casa Pastor in King's Cross
2. Tayyabs in Whitechapel
3. Oklava in Shoreditch
4. Bright in Hackney
5. Fish, Wings & Tings in Brixton
6. Kudu in Peckham
7. Petersham Nurseries Café and Tearoom in Richmond
8. Maggie Jones's in Kensington

You can find more information about these restaurants and other local food spots in London on the [Visit London website](https://www.visitlondon.com/things-to-do/food-and-drink/restaurant/local-restaurants).

Please note that this information is based on the available sources and may be subject to change.
